In [1]:
import numpy as np
import pandas as pd
from datetime import date

In [3]:
def load_data(filename):
    return pd.read_csv(filename, sep = ',', parse_dates=['Date'])

In [2]:
def prepared_data(df):
    # Уберем столбцы, где более половины значений отсутствуют
    na_50 = np.where(df.isna().sum() > len(df) / 2)
    cols_na_50 = df.columns[na_50]
    df = df.drop(cols_na_50, axis=1)
    
    # В большинство колонок типа object недостающие значения
    # можно заполнить как Undefined типа string
    obj_names = df.columns[np.where(df.dtypes == 'object')]
    
    # Столбец Date является типом дата
    # Недостающие значения можно заполнить текущей датой
    df.Date = pd.to_datetime(df.Date.astype('datetime64'))
    if df.Date.isna().sum() != 0:
        df['Date'] = df['Date'].fillna(date.today()).astype('datetime64')
    
    # Заполнение и преобразование строковых значений
    df[obj_names] = df[obj_names].fillna('Undefined').astype('string')
    
    # Обработка бинарных признаков
    bin_values = []
    for col in df.columns:
        if len(np.array(df[col].unique())) <= 3: # True, False, Undefined
            bin_values.append(col)
    
    mapping = {
        'No': 0,
        'Yes': 1,
        'Undefined': 0
    }

    for val in bin_values:
        df[val] = df[val].map(mapping)
        
    # Обработка категориальных признаков
    # Если в столбце более 15 значений - удаляем
    string_cols = df.columns[np.where(df.dtypes == 'string')]
    drop_values = []
    dummies_values = []

    for col in string_cols:
        if len(df[col].unique()) > 15:
            drop_values.append(col)
        else:
            dummies_values.append(col)
            
    df = df.drop(drop_values, axis=1)
    df = pd.get_dummies(df, columns=dummies_values)
    
    # Заполнение оставшихся пропусков средними значениями
    df = df.fillna(df.mean(numeric_only=True))
    
    return df

In [ ]:
students_df = load_data('../data/students.csv')
students_df = prepared_data(students_df)